# Segmentation de tumeurs cérébrales sur IRM avec U-Net  

### **Date** : Janvier 2026  
### **Objectif** : Construire, entraîner et évaluer un modèle U-Net pour la segmentation de tumeurs sur images IRM cérébrales  
### **Niveau de complexité** : Intermédiaire → Avancé (compréhension de la segmentation médicale)
### **Réallisé par** :
####      Benzekri Inssaf
####      Nathan Kabassele 
####     Ahmed Jabri
---

## 1️⃣ Importation des bibliothèques

Cette cellule importe toutes les bibliothèques nécessaires au projet :

- **TensorFlow / Keras** : construction et entraînement du modèle Deep Learning  
- **NumPy** : manipulation des tableaux numériques  
- **Matplotlib** : visualisation des images et des courbes  
- **OpenCV (cv2)** : sauvegarde et traitement d’images  
- **Scikit-learn** : séparation des données (train / validation / test)  

On affiche également la version de TensorFlow pour assurer la compatibilité.


In [ ]:
# =============================================
# 1. IMPORTATIONS
# =============================================
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import zipfile
import gdown
from sklearn.model_selection import train_test_split

print("✅ Importations terminées")
print(f"TensorFlow version: {tf.__version__}")

## 2️⃣ Préparation du dataset (données synthétiques)

Dans un contexte pédagogique, nous générons un **dataset synthétique** simulant des images IRM cérébrales :

### 🔹 Pourquoi des données synthétiques ?
- Éviter la dépendance à Kaggle / Internet
- Faciliter les tests rapides
- Comprendre le pipeline sans complexité médicale

### 🔹 Description
- Images en niveaux de gris (128×128)
- Ajout de tumeurs artificielles sous forme de cercles
- Création des masques de segmentation correspondants
- Ajout de bruit réaliste

Chaque image est sauvegardée pour permettre une inspection visuelle.



In [ ]:
print("\n📥 Téléchargement du dataset depuis Kaggle...")

# Dataset: Brain MRI Segmentation (petit dataset pour test)
dataset_url = "https://www.kaggle.com/datasets/mateuszbuda/lgg-mri-segmentation"
# Alternative: Télécharger directement depuis Google Drive
drive_url = "https://drive.google.com/uc?id=1c0Lx4XSD8XNy1bOe3sS0MfywxHuh-cDr"

dataset_path = "brain_mri_dataset.zip"
extract_path = "brain_mri_data"

# Créer un dataset synthétique si le téléchargement échoue
print("⚠️  Création de données synthétiques pour la démonstration...")

# Créer des dossiers pour les données synthétiques
os.makedirs("synthetic_data/images", exist_ok=True)
os.makedirs("synthetic_data/masks", exist_ok=True)

# Générer 100 images synthétiques (IRM) et masques
num_samples = 100
img_size = 128

X_synthetic = []
y_synthetic = []

for i in range(num_samples):
    # Image IRM synthétique (niveaux de gris avec structures)
    img = np.random.randn(img_size, img_size) * 0.1 + 0.5

    # Ajouter des "tumeurs" synthétiques (cercles aléatoires)
    mask = np.zeros((img_size, img_size))

    # Nombre aléatoire de tumeurs (0-3)
    num_tumors = np.random.randint(0, 4)
    for _ in range(num_tumors):
        center_x = np.random.randint(20, img_size-20)
        center_y = np.random.randint(20, img_size-20)
        radius = np.random.randint(5, 15)

        # Créer un cercle pour la tumeur
        y, x = np.ogrid[-center_y:img_size-center_y, -center_x:img_size-center_x]
        tumor_mask = x*x + y*y <= radius*radius
        mask[tumor_mask] = 1

        # Ajouter un effet sur l'image IRM
        img[tumor_mask] = img[tumor_mask] * 0.7  # Plus sombre

    # Ajouter du bruit
    img = img + np.random.normal(0, 0.05, (img_size, img_size))
    img = np.clip(img, 0, 1)

    # Ajouter les canaux
    img = np.expand_dims(img, axis=-1)
    mask = np.expand_dims(mask, axis=-1)

    X_synthetic.append(img)
    y_synthetic.append(mask)

    # Sauvegarder pour visualisation
    cv2.imwrite(f"synthetic_data/images/mri_{i:03d}.png", (img[:,:,0]*255).astype(np.uint8))
    cv2.imwrite(f"synthetic_data/masks/mask_{i:03d}.png", (mask[:,:,0]*255).astype(np.uint8))

X = np.array(X_synthetic, dtype=np.float32)
y = np.array(y_synthetic, dtype=np.float32)

print(f"✅ Dataset synthétique créé: {len(X)} images")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"Pourcentage moyen de tumeurs: {np.mean(y)*100:.2f}%")

## 3️⃣ Visualisation des données

Cette cellule permet d’explorer visuellement le dataset :

Pour chaque échantillon affiché :
- 🧠 Image IRM originale
- 🎯 Masque de vérité terrain (ground truth)
- 🧩 Superposition image + masque
- 📊 Histogramme des intensités de pixels

Objectif :
> Vérifier la cohérence des données avant l’entraînement.


In [ ]:
print("\n👁️ Visualisation de quelques échantillons...")

fig, axes = plt.subplots(3, 4, figsize=(12, 9))

for i in range(3):
    idx = np.random.randint(0, len(X))

    # Image IRM
    axes[i, 0].imshow(X[idx, :, :, 0], cmap='gray')
    axes[i, 0].set_title(f'IRM {idx}')
    axes[i, 0].axis('off')

    # Masque de vérité
    axes[i, 1].imshow(y[idx, :, :, 0], cmap='gray')
    axes[i, 1].set_title(f'Masque {idx}')
    axes[i, 1].axis('off')

    # Superposition
    overlay = X[idx, :, :, 0].copy()
    overlay = np.stack([overlay]*3, axis=-1)
    overlay[y[idx, :, :, 0] > 0.5, 1] = 1  # Vert pour les tumeurs

    axes[i, 2].imshow(overlay)
    axes[i, 2].set_title(f'Superposition {idx}')
    axes[i, 2].axis('off')

    # Histogramme des valeurs
    axes[i, 3].hist(X[idx].flatten(), bins=50, alpha=0.7)
    axes[i, 3].set_title(f'Histogramme {idx}')
    axes[i, 3].set_xlabel('Intensité')
    axes[i, 3].set_ylabel('Fréquence')

plt.tight_layout()
plt.show()


## 4️⃣ Division du dataset

Les données sont divisées comme suit :
- **70 %** entraînement
- **15 %** validation
- **15 %** test

Pourquoi cette séparation ?
- Le modèle apprend sur le train set
- Les hyperparamètres sont ajustés via le validation set
- Les performances finales sont mesurées sur le test set

On affiche également le **pourcentage moyen de tumeurs** dans chaque sous-ensemble.


In [ ]:
print("\n📊 Division des données...")

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Train set: {len(X_train)} images")
print(f"Validation set: {len(X_val)} images")
print(f"Test set: {len(X_test)} images")
print(f"Pourcentage tumeurs (train): {np.mean(y_train)*100:.2f}%")
print(f"Pourcentage tumeurs (val): {np.mean(y_val)*100:.2f}%")
print(f"Pourcentage tumeurs (test): {np.mean(y_test)*100:.2f}%")

## 5️⃣ Construction du modèle U-Net

Le modèle utilisé est un **U-Net**, architecture très populaire en segmentation médicale.

### 🔹 Architecture
- **Encoder** : extraction progressive de caractéristiques
- **Bottleneck** : représentation compacte
- **Decoder** : reconstruction spatiale
- **Skip connections** : préservation des détails fins

### 🔹 Fonctions personnalisées
- **Dice coefficient** : mesure de similarité entre masques
- **IoU (Intersection over Union)** : métrique clé en segmentation

### 🔹 Compilation
- Optimiseur : Adam
- Fonction de perte : Binary Crossentropy
- Métriques : Accuracy, IoU, Dice


In [ ]:
print("\n🤖 Construction du modèle U-Net...")

from tensorflow.keras import layers, Model

# Définition des métriques
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

def iou_metric(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    union = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

# Construction du modèle
inputs = layers.Input(shape=(128, 128, 1))

# Encoder
c1 = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
c1 = layers.Conv2D(32, 3, activation='relu', padding='same')(c1)
p1 = layers.MaxPooling2D((2, 2))(c1)

c2 = layers.Conv2D(64, 3, activation='relu', padding='same')(p1)
c2 = layers.Conv2D(64, 3, activation='relu', padding='same')(c2)
p2 = layers.MaxPooling2D((2, 2))(c2)

c3 = layers.Conv2D(128, 3, activation='relu', padding='same')(p2)
c3 = layers.Conv2D(128, 3, activation='relu', padding='same')(c3)
p3 = layers.MaxPooling2D((2, 2))(c3)

# Centre
c4 = layers.Conv2D(256, 3, activation='relu', padding='same')(p3)
c4 = layers.Conv2D(256, 3, activation='relu', padding='same')(c4)

# Decoder
u5 = layers.Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(c4)
u5 = layers.concatenate([u5, c3])
c5 = layers.Conv2D(128, 3, activation='relu', padding='same')(u5)
c5 = layers.Conv2D(128, 3, activation='relu', padding='same')(c5)

u6 = layers.Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(c5)
u6 = layers.concatenate([u6, c2])
c6 = layers.Conv2D(64, 3, activation='relu', padding='same')(u6)
c6 = layers.Conv2D(64, 3, activation='relu', padding='same')(c6)

u7 = layers.Conv2DTranspose(32, 2, strides=(2, 2), padding='same')(c6)
u7 = layers.concatenate([u7, c1])
c7 = layers.Conv2D(32, 3, activation='relu', padding='same')(u7)
c7 = layers.Conv2D(32, 3, activation='relu', padding='same')(c7)

outputs = layers.Conv2D(1, 1, activation='sigmoid')(c7)

model = Model(inputs=inputs, outputs=outputs)

# Compilation
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', iou_metric, dice_coef]
)

model.summary()

## 6️⃣ Entraînement du modèle

Le modèle est entraîné sur les données d’apprentissage avec :

### 🔹 Callbacks utilisés
- **EarlyStopping** : évite le sur-apprentissage
- **ReduceLROnPlateau** : ajuste automatiquement le learning rate

### 🔹 Paramètres
- Epochs : 20
- Batch size : 8

À la fin de l’entraînement, le modèle est sauvegardé au format `.keras`.


In [ ]:
print("\n🚀 Début de l'entraînement...")

# Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3
    )
]

# Entraînement
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=8,
    callbacks=callbacks,
    verbose=1
)

# Sauvegarde
model.save('brain_tumor_model.keras')
print("✅ Modèle sauvegardé: brain_tumor_model.keras")  

## 7️⃣ Visualisation de l’apprentissage

Cette cellule affiche les courbes suivantes :
- 📉 Loss (train / validation)
- ✅ Accuracy
- 📐 IoU
- 🎯 Dice coefficient

Objectif :
> Vérifier la convergence du modèle et détecter un éventuel overfitting.


In [ ]:
print("\n📈 Visualisation de l'entraînement...")

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Loss
axes[0, 0].plot(history.history['loss'], label='Train Loss')
axes[0, 0].plot(history.history['val_loss'], label='Val Loss')
axes[0, 0].set_title('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Accuracy
axes[0, 1].plot(history.history['accuracy'], label='Train Accuracy')
axes[0, 1].plot(history.history['val_accuracy'], label='Val Accuracy')
axes[0, 1].set_title('Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True)

# IoU
axes[1, 0].plot(history.history['iou_metric'], label='Train IoU')
axes[1, 0].plot(history.history['val_iou_metric'], label='Val IoU')
axes[1, 0].set_title('IoU Metric')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Dice
axes[1, 1].plot(history.history['dice_coef'], label='Train Dice')
axes[1, 1].plot(history.history['val_dice_coef'], label='Val Dice')
axes[1, 1].set_title('Dice Coefficient')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

## 8️⃣ Évaluation du modèle sur le test set

Cette étape mesure les performances réelles du modèle sur des données jamais vues :

### 🔹 Métriques calculées
- IoU moyen
- Dice moyen
- Précision
- Rappel
- F1-score

Une analyse de **sur-segmentation** ou **sous-segmentation** est également réalisée.


In [ ]:
print("\n📊 Évaluation sur le test set...")

# Prédictions
y_pred = model.predict(X_test, verbose=1)
y_pred_binary = (y_pred > 0.5).astype(np.float32)

# Calcul des métriques
iou_scores = []
dice_scores = []
precision_scores = []
recall_scores = []

for i in range(len(X_test)):
    iou = iou_metric(y_test[i], y_pred_binary[i]).numpy()
    dice = dice_coef(y_test[i], y_pred_binary[i]).numpy()

    y_true_f = y_test[i].flatten()
    y_pred_f = y_pred_binary[i].flatten()

    tp = np.sum((y_true_f == 1) & (y_pred_f == 1))
    fp = np.sum((y_true_f == 0) & (y_pred_f == 1))
    fn = np.sum((y_true_f == 1) & (y_pred_f == 0))

    precision = tp / (tp + fp + 1e-6)
    recall = tp / (tp + fn + 1e-6)

    iou_scores.append(iou)
    dice_scores.append(dice)
    precision_scores.append(precision)
    recall_scores.append(recall)

print("\n" + "="*50)
print("📊 RÉSULTATS FINAUX")
print("="*50)
print(f"IoU moyen: {np.mean(iou_scores):.4f} (+/- {np.std(iou_scores):.4f})")
print(f"Dice moyen: {np.mean(dice_scores):.4f} (+/- {np.std(dice_scores):.4f})")
print(f"Précision moyenne: {np.mean(precision_scores):.4f}")
print(f"Rappel moyen: {np.mean(recall_scores):.4f}")
print(f"F1-Score: {2 * np.mean(precision_scores) * np.mean(recall_scores) / (np.mean(precision_scores) + np.mean(recall_scores) + 1e-6):.4f}")

# Statistiques
avg_tumor_actual = np.mean(y_test) * 100
avg_tumor_pred = np.mean(y_pred_binary) * 100
print(f"\n📈 STATISTIQUES:")
print(f"  Tumeurs réelles moyennes: {avg_tumor_actual:.1f}%")
print(f"  Tumeurs prédites moyennes: {avg_tumor_pred:.1f}%")
print(f"  Différence: {abs(avg_tumor_actual - avg_tumor_pred):.1f}%")

if avg_tumor_pred > avg_tumor_actual:
    print(f"  ⚠️  SUR-SEGMENTATION: {avg_tumor_pred - avg_tumor_actual:.1f}% de faux positifs")
elif avg_tumor_pred < avg_tumor_actual:
    print(f"  ⚠️  UNDER-SEGMENTATION: {avg_tumor_actual - avg_tumor_pred:.1f}% de faux négatifs")


## 9️⃣ Visualisation des prédictions

Pour plusieurs images aléatoires du test set :
- Image IRM originale
- Masque réel
- Masque prédit
- Superposition couleur :
  - 🟢 Vert : vérité terrain
  - 🔴 Rouge : prédiction
  - 🟡 Jaune : prédiction correcte

Chaque image affiche également son score IoU.


In [ ]:
print("\n👁️ Visualisation des prédictions...")

# Sélectionner 3 exemples aléatoires
indices = np.random.choice(len(X_test), 3, replace=False)

fig, axes = plt.subplots(3, 4, figsize=(15, 10))

for i, idx in enumerate(indices):
    # Image originale
    img = X_test[idx, :, :, 0]

    # Vérité terrain
    true_mask = y_test[idx, :, :, 0]

    # Prédiction
    pred = y_pred[idx, :, :, 0]
    pred_binary = (pred > 0.5).astype(np.float32)

    # Calcul IoU
    iou = iou_metric(true_mask, pred_binary).numpy()

    # Image IRM
    axes[i, 0].imshow(img, cmap='gray')
    axes[i, 0].set_title(f'IRM {idx}')
    axes[i, 0].axis('off')

    # Vérité terrain
    axes[i, 1].imshow(true_mask, cmap='gray')
    axes[i, 1].set_title(f'Vérité {idx}')
    axes[i, 1].axis('off')

    # Prédiction binaire
    axes[i, 2].imshow(pred_binary, cmap='gray')
    axes[i, 2].set_title(f'Prédiction\nIoU: {iou:.3f}')
    axes[i, 2].axis('off')

    # Superposition
    overlay = np.stack([img]*3, axis=-1)
    overlay[true_mask > 0.5, 1] = 1  # Vert pour vérité
    overlay[pred_binary > 0.5, 0] = 1  # Rouge pour prédiction
    overlay[(true_mask > 0.5) & (pred_binary > 0.5), 0] = 1  # Jaune pour correct
    overlay[(true_mask > 0.5) & (pred_binary > 0.5), 1] = 1

    axes[i, 3].imshow(overlay)
    axes[i, 3].set_title('Superposition\n(Vert=Vrai, Rouge=Prédit)')
    axes[i, 3].axis('off')

plt.tight_layout()
plt.show()

## 🔟 Analyse détaillée d’un cas test

Cette cellule analyse un exemple précis du test set :

- Comparaison pixel par pixel
- Carte de confiance (probabilités)
- Carte d’erreur
- Calcul précis des métriques

Objectif :
> Comprendre **où et pourquoi** le modèle se trompe.


In [ ]:
print("\n🔍 Test détaillé sur un exemple...")

test_idx = 0  # Premier exemple du test set
test_img = X_test[test_idx]
true_mask = y_test[test_idx]

# Prédiction
pred = model.predict(test_img[np.newaxis, ...], verbose=0)[0]
pred_binary = (pred > 0.5).astype(np.float32)

# Métriques
iou = iou_metric(true_mask, pred_binary).numpy()
dice = dice_coef(true_mask, pred_binary).numpy()

print(f"\n📊 Métriques pour l'exemple {test_idx}:")
print(f"IoU: {iou:.4f}")
print(f"Dice: {dice:.4f}")
print(f"Tumeurs réelles: {np.mean(true_mask)*100:.1f}%")
print(f"Tumeurs prédites: {np.mean(pred_binary)*100:.1f}%")

# Visualisation détaillée
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

# Image IRM
axes[0, 0].imshow(test_img[:, :, 0], cmap='gray')
axes[0, 0].set_title('IRM Originale')
axes[0, 0].axis('off')

# Vérité terrain
axes[0, 1].imshow(true_mask[:, :, 0], cmap='gray')
axes[0, 1].set_title('Vérité Terrain')
axes[0, 1].axis('off')

# Prédiction
axes[0, 2].imshow(pred_binary[:, :, 0], cmap='gray')
axes[0, 2].set_title(f'Prédiction (IoU: {iou:.3f})')
axes[0, 2].axis('off')

# Différence
diff = np.abs(true_mask[:, :, 0] - pred_binary[:, :, 0])
axes[1, 0].imshow(diff, cmap='hot')
axes[1, 0].set_title('Différence')
axes[1, 0].axis('off')

# Carte de confiance
axes[1, 1].imshow(pred[:, :, 0], cmap='viridis')
axes[1, 1].set_title('Carte de Confiance')
axes[1, 1].axis('off')

# Superposition
overlay = np.stack([test_img[:, :, 0]]*3, axis=-1)
overlay[true_mask[:, :, 0] > 0.5, 1] = 0.7
overlay[pred_binary[:, :, 0] > 0.5, 0] = 0.7
axes[1, 2].imshow(overlay)
axes[1, 2].set_title('Superposition')
axes[1, 2].axis('off')

plt.suptitle(f'Analyse Détaillée - Exemple {test_idx}', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 1️⃣1️⃣ Analyse des erreurs du modèle

Cette dernière section :
- Identifie la **meilleure** et la **pire** prédiction
- Affiche la distribution des scores IoU
- Fournit un résumé global des performances

Cette analyse est essentielle pour :
- améliorer le modèle,
- ajuster le seuil,
- ou enrichir les données.


In [ ]:
print("\n🔬 Analyse des erreurs...")

# Trouver les meilleurs et pires prédictions
all_ious = [iou_metric(y_test[i], (model.predict(X_test[i:i+1]) > 0.5).astype(np.float32)).numpy()
            for i in range(len(X_test))]

best_idx = np.argmax(all_ious)
worst_idx = np.argmin(all_ious)

print(f"\n🎯 Meilleure prédiction: index {best_idx}, IoU = {all_ious[best_idx]:.4f}")
print(f"⚠️  Pire prédiction: index {worst_idx}, IoU = {all_ious[worst_idx]:.4f}")

# Histogramme des IoU
plt.figure(figsize=(10, 5))
plt.hist(all_ious, bins=20, edgecolor='black', alpha=0.7)
plt.axvline(x=np.mean(all_ious), color='red', linestyle='--', label=f'Moyenne: {np.mean(all_ious):.3f}')
plt.xlabel('IoU Score')
plt.ylabel('Fréquence')
plt.title('Distribution des Scores IoU sur le Test Set')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\n" + "="*50)
print("✅ NOTEBOOK TERMINÉ AVEC SUCCÈS!")
print("="*50)
print(f"📊 Résumé des performances:")
print(f"   - IoU moyen: {np.mean(all_ious):.3f}")
print(f"   - Meilleur IoU: {np.max(all_ious):.3f}")
print(f"   - Pire IoU: {np.min(all_ious):.3f}")
print(f"   - Modèle sauvegardé: brain_tumor_model.keras")